<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>






# Neighboorhood and Spatial Correlation.



## Getting ready


### Libraries needed

Let's verify:

In [ ]:
!pip show pysal pandas geopandas

In [ ]:
## needed in Colab
# !pip install pysal

### Data to use

Let me get two maps:

1. The USA map, at states level,  directly from census.gov, which has a good quality.

In [ ]:
import geopandas as gpd

url = "https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_500k.zip"
us_states = gpd.read_file(url)
us_states.info(),us_states.crs.to_epsg(),us_states.crs.is_projected

Notice this map has basic information per state. Also, notice the current crs will plot this:

In [ ]:
us_states.plot()

Let's reproject this map:

In [ ]:
us_states=us_states.to_crs(5070)
us_states.plot()

Let's use the state name as index, that would help an easier identification of the places when we see most outputs (otherwise we will see just numerical indexes) :

In [ ]:
us_states.set_index('NAME', inplace=True)
us_states.head()

Let's subset the us_states for some examples:

In [ ]:
someStates=['Utah','Colorado','Arizona','New Mexico', 'Florida','Georgia','Alabama']
sub_us=us_states[us_states.index.isin(someStates)]
sub_us

2. A map of Peru, at the 'distrito' level (similar to municipality in the USA - not exactly the same). The map comes from an unoffical [website](https://www.geogpsperu.com/p/descargas.html). Some columns have been added.

In [ ]:
peruDataLink="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/PERU/PeruMaps.gpkg"
peru_distritos=gpd.read_file(peruDataLink,layer='distritos')

# some basic info
peru_distritos.info(),peru_distritos.crs.to_epsg(),peru_distritos.crs.is_projected

Let's reproject and plot:

In [ ]:
peru_distritos=peru_distritos.to_crs(5387)
peru_distritos.plot()

Besides the spatial units (DEPARTAMEN, PROVINCIA, DISTRITO, and Ubigeo - "Ubigeo" is a code ), you have:
 - **Poblacion**: Population (2017)
 - **Superficie**: Area               
 - **IDH2019**: Human Development Index for DISTRITO (2019)                   
 - **Educ_sec_comp2019_pct**: Share of Population that finished High-School (2019)     
 - **NBI2017_pct**: Share of Population with poverty at the household level aggregated by DISTRITO. This index ("Unsatisfied Basic Needs") uses observable living conditions rather than income alone (2017).
 - **Viv_sin_serv_hig2017_pct**: Share of housing units that have no sanitation infrastructure aggregated by  DISTRITO (2017)

Notice we should not use the 'distrito' name as index, because several of them are repeated:

In [ ]:
peru_distritos[peru_distritos['DISTRITO'].duplicated()]

Let's use 'Ubigeo', although is not the best solution

In [ ]:
# of course
peru_distritos[peru_distritos['Ubigeo'].duplicated()]

In [ ]:
#then
peru_distritos.set_index('Ubigeo', inplace=True)

## I. Who is my neighbor?

In spatial analysis, the intuitive concept of a “neighbor” can be operationalized in multiple ways.

So far, we have identified neighbors using geometric operations such as buffering, spatial joins, and overlays. Now, let’s consider the distance matrix:


In [ ]:
sub_us.geometry.apply\
(lambda state: sub_us.distance(state)/1000)

In [ ]:
sub_us.explore()

From this matrix and plot, you’ll notice that neighboring features have a distance of zero—this occurs when two polygons share a boundary (i.e., they are contiguous). This observation helps illustrate the different approaches to defining neighbors:

1. Binary Relationships:

* Contiguity: Two polygons are considered neighbors if they share any portion of their boundary—whether at a point, along a line segment, or more extensively. In the distance matrix, such pairs show a distance of zero, reflecting direct spatial adjacency.
* Discrete Proximity: In this approach, you request K neighbors, and you get the K closest ones to you.


2. Continuous Relationships

* Proximity: Two features are considered neighbors if the distance between them falls below a specified threshold (e.g., within 100 km). This approach is especially useful for point data or non-contiguous regions.
* Shared Border Length: The strength of the neighbor relationship is weighted by the length of the shared boundary. Longer shared borders imply stronger spatial interaction—a common assumption in models of spatial diffusion or economic spillovers.

Using matrices—rather than raw geometries—is essential for the mathematical representation and numerical computation required in upcoming spatial analytics techniques.

PySAL (libpysal) is designed to handle spatial relationship matrices and integrates seamlessly with GeoPandas. Rather than relying solely on matrices, modern PySAL uses graph-based representations of spatial relationships. This approach is not only more memory-efficient but also speeds up computation and simplifies visualization—especially for large or sparse spatial datasets.  A key concept in these graphs is the distinction between:

- Focal unit: the spatial feature (e.g., a state, county, or census tract) for which we are identifying neighbors.
- Neighbor(s): the other spatial units that are considered related to the focal unit based on a chosen criterion (e.g., contiguity, distance, or shared border length).

Let’s become familiar with the ```graph``` module in PySAL/libpysal, which provides a modern and flexible framework for constructing and working with spatial neighbor graphs.


In [ ]:
from libpysal.graph import Graph

### I. 1 Contiguity and Binary matrices

Take a look at the **queen** and **rook** relationship:

<center><img src="https://github.com/CienciaDeDatosEspacial/spatial_autoCorr/raw/main/rookQueen.png" width="700"></center>

From the image above:
- Your **rook** neighbor is whichever  shares a border with you (a borderline of at least two points). It is also known as the _Von Neumann_ neighbor.

- Your **queen** neighbor is whichever  shares a border or a corner with you (at least one point).It is also known as the _Moore_ neighbor.

Let's see how to get each set of neighbors

#### I.1.a Rook

* The key idea: A focal polygon considers another polygon a neighbor if they share a common edge—that is, two or more connected points forming a line segment of positive length.
* The input: A GeoDataFrame (GDF) containing polygon geometries (e.g., states, counties, census tracts). Each row represents a spatial unit that can serve as a focal observation.
* The process: For each polygon (focal unit), the algorithm checks all other polygons to determine whether their boundaries intersect along a line segment (not just at a point).
* The output: A binary spatial graph (or adjacency structure) where each node represents a polygon (focal unit). An edge exists between two nodes only if they share a boundary segment. The corresponding adjacency matrix is binary.

Given our input for the examples is **sub_us**, let's run...

In [ ]:
sub_us_rook=Graph.build_contiguity(sub_us,rook=True)

Now let's check the ouput:

**a. adjacency**

In [ ]:
sub_us_rook.adjacency

The previous results shows only the neighbors of the focals, to recreate a wide format:

In [ ]:
import pandas as pd
pd.DataFrame(sub_us_rook.adjacency).unstack()

We generally fill those missing valiues (not a neighbor) with zero.

In [ ]:
pd.DataFrame(sub_us_rook.adjacency).unstack().fillna(0)

**b. adjacency graph**

As we have a `GRAPH`, we can identify these neighborhood relationships via edges:

In [ ]:
sub_us_rook.explore(us_states, edge_kws=dict(alpha=0.4),zoom_start = 6)

#### I.1.b Queen

* The key idea: A focal polygon considers another polygon a neighbor if they share a common edge or a vertex (at least one point).
* The input: A GeoDataFrame (GDF) containing polygon geometries (e.g., states, counties, census tracts). Each row represents a spatial unit that can serve as a focal observation.
* The process: For each polygon (focal unit), the algorithm checks all other polygons to determine whether their boundaries intersect at any point or along a line segment.
* The output: A binary spatial graph (or adjacency structure) where each node represents a polygon (focal unit). An edge exists between two nodes only if they share a point or boundary segment. The corresponding adjacency matrix is binary.

Let's see what we get:

In [ ]:
# first
sub_us_queen=Graph.build_contiguity(sub_us,rook=False)

**a. adjacency**

In [ ]:
pd.DataFrame(sub_us_queen.adjacency).unstack().fillna(0)

In [ ]:
sub_us_queen.explore?

In [ ]:
sub_us_queen.explore(us_states, edge_kws=dict(alpha=0.4),zoom_start = 6)



### I. 2 Disctrete proximity and Binary matrices


### KNN Proximity

* **The key idea**: focal wants an amount 'K' of neighbors.
* **The input**: the amount K of neighbors desired; the GDF of centroids or 'representative points' for every focal.
* **The process**: for each node (focal) compute distances to the rest of nodes, so as to keep the K of closest nodes.
* **The output**: a graph, where each node has at most K edges to the K closest nodes ('neighbors'). Edges are represented by a dichotomous adjacency matrix.

Here it is:

In [ ]:
sub_us_knn3 = Graph.build_knn(sub_us.representative_point(), # GDF
                                 k=3) # desired k
sub_us_knn3.adjacency

In [ ]:
adj_matrix = pd.DataFrame(sub_us_knn3.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight',fill_value=0
)

adj_matrix_wide

In [ ]:
sub_us_knn3.explore(us_states, edge_kws=dict(alpha=0.4))

### Discrete Distance band-based Proximity

* **The key idea**: focal wants all neighbors within a radius distance.
* **The input**: the threshold (radius); the GDF of centroids or 'representative points' for every focal.
* **The process**: for each node (focal) compute distances to all other nodes, keeping only those within the radius.
* **The output**: a graph where each node has edges to all nodes ('neighbors') whose representative points lie within the radius. Edges are represented by a dichotomous adjacency matrix.

Here it is:

In [ ]:
us_states_band350k_D=Graph.build_distance_band(sub_us.representative_point(), threshold=350000)
us_states_band350k_D.adjacency

In [ ]:
adj_matrix = pd.DataFrame(us_states_band350k_D.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight',fill_value=0
)

adj_matrix_wide

In [ ]:
us_states_band350k_D.explore(us_states, edge_kws=dict(alpha=0.4))

### Continuos Distance band-based Proximity

* **The key idea**: focal wants all neighbors within a radius distance.
* **The input**: the threshold (radius); the GDF of centroids or 'representative points' for every focal.
* **The process**: for each node (focal) compute distances to all other nodes, keeping only those within the radius.
* **The output**: a graph where each node has edges to all nodes ('neighbors') whose representative points lie within the radius. Edges are represented by a continuous adjacency matrix, the values represent the inverse distance between the nodes in the edges.

Here it is:

In [ ]:
sub_us_band750k_C=Graph.build_distance_band(sub_us.representative_point(), threshold=750000,binary=False)
sub_us_band750k_C.adjacency


In [ ]:
adj_matrix = pd.DataFrame(sub_us_band750k_C.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight',fill_value=0
)

adj_matrix_wide

In [ ]:
sub_us_band750k_C.explore(
        us_states, edge_kws=dict(alpha=0.4)
    )

In [ ]:
sub_us_kernel5 = Graph.build_kernel(sub_us.representative_point(), k=5)
sub_us_kernel5.adjacency

In [ ]:
adj_matrix = pd.DataFrame(sub_us_kernel5.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight',fill_value=0
)

adj_matrix_wide

In [ ]:
sub_us_kernel5.explore(
        us_states, edge_kws=dict(column="weight",
        style_kwds=dict(weight=5)    )
    )

In [ ]:
sub_us_perimeter = Graph.build_contiguity(sub_us, by_perimeter=True)
sub_us_perimeter.adjacency


In [ ]:
adj_matrix = pd.DataFrame(sub_us_perimeter.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight',fill_value=0
)

adj_matrix_wide

In [ ]:
sub_us_perimeter.explore(
        us_states, edge_kws=dict(column="weight",
        style_kwds=dict(weight=5)    )
    )

In [ ]:
us_states_rook.isolates

We can know several interesting information from this output:

1. Count of nodes, count of rook neighbor relationships

In [ ]:
us_states_rook.n, us_states_rook.n_edges/2

2. For each state(focal node) which is a neighbor?

In [ ]:
us_states_rook.adjacency

3. For each state(focal node) how many neighbors does he have?

In [ ]:
us_states_rook.cardinalities

We can see a distribution of the cardinalities like this:

In [ ]:
us_states_rook.cardinalities.plot(kind='hist')

### The QUEEN neighbors

The same function can give us the Queen neigbors:

In [ ]:
us_states_queen=Graph.build_contiguity(us_states,rook=False)

If there are not adjacent polygons , they will still be isolates:

In [ ]:
us_states_queen.isolates

Compare the cardinalities:

In [ ]:
us_states_rook.cardinalities.rename('rook').plot(kind='hist',legend=True,alpha=0.6)
us_states_queen.cardinalities.rename('queen').plot(kind='hist',legend=True,alpha=0.6)



In general, when we request queen neighbors, you would see more values on the rigth tail, and some more flatteing (dispersion).

Let's confirm the difference (and the quality of the map)

In [ ]:
check=['Colorado']

us_states_rook.adjacency.loc[check]

In [ ]:
us_states_queen.adjacency.loc[check]

Arizona is a queen neighbor only. This difference is due to the Four Corners Monument where Colorado meets Arizona at a single point. Our ap seems to have pretty good quality!

### The KNN neighbors

This is a total different idea. Here you get the actual 'K" neighbors of any focal point. Adjacency is not a problem, only proximity.
Let me compute the 8 closest neighbosr for every state:

In [ ]:
# K= 8
us_states_knn8 = Graph.build_knn(us_states.representative_point(), k=8)

Let's plot the result:

In [ ]:
# K= 8


You see no isolates this time, each state has 8 neighbors.

This plot confirms the lack of variability:

In [ ]:
us_states_knn8.cardinalities.plot(kind='hist',alpha=0.6)

In [ ]:
us_states_knn8.adjacency

### Kerneling

In [ ]:
us_states_kernel8 = Graph.build_kernel(us_states.representative_point(), k=8)


In [ ]:
us_states_kernel8.adjacency

In [ ]:
us_states_dist_1k = Graph.build_distance_band(us_states.representative_point(), 1000000, binary=False)


In [ ]:
us_states_dist_1k.adjacency

In [ ]:
import pandas as pd

# Reshape from long to wide format
adj_matrix = pd.DataFrame(us_states_kernel8.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight'
)

adj_matrix_wide

$$w_{ij} = \dfrac{1}{d_{ij}}$$

In [ ]:

us_states_perimeter = Graph.build_contiguity(us_states, by_perimeter=True)


In [ ]:
us_states_perimeter.adjacency

## Am I independent from my neighbors?

This is a crucial moment is statistics: traditionally, we assume our situation is independent of others'; so sampling may reveal unbiased  population insights. But, what if neigbors are affecting one another.

Of course, by situation we mean a variable, for example, average HS completed in my 'distrito':

In [ ]:
peru_distritos.Educ_sec_comp2019_pct.describe()

See the choropleth:

In [ ]:
peru_distritos.plot(
    "Educ_sec_comp2019_pct",
    scheme="quantiles",
    cmap="Reds_r",
    legend=True,figsize=(12, 10))

The job now, is to compute the average of my neighbors on this variable.

### Adjacency and weights


We know by now that we have to decide one neighborhood approach. Then, let's compute the queen neighbors:

In [ ]:
peru_distritos_queen=Graph.build_contiguity(peru_distritos,rook=False)

Now, we have the neighbors:

In [ ]:
peru_distritos_queen.adjacency

Let me turn that into a 'wide shape':

In [ ]:
import pandas as pd

# Reshape from long to wide format
adj_matrix = pd.DataFrame(peru_distritos_queen.adjacency).reset_index()
adj_matrix_wide = adj_matrix.pivot_table(
    index='focal',
    columns='neighbor',
    values='weight',
    fill_value=0
)

adj_matrix_wide

Of course, you could get the cardinalities this way:

In [ ]:
adj_matrix_wide.sum(axis=1)

But we do not need cardinalities here. If needed it, I will simply be able to compute the average

What I need is to use this neighborhood as a way to weight their effect on my situation.



You can compute the adjacency with the weights now:

In [ ]:
peru_distritos_queen=peru_distritos_queen.transform("r")

In [ ]:
The way to know if my value (education level) depends on my neigbor's, is to represent each 'distrito" by the average of my neighbor values.


Now, that weighted mean is the **lagged** variables:

In [ ]:
y = peru_distritos["Educ_sec_comp2019_pct"]
ylag = peru_distritos_queen.lag(y)

Let me add it to the GDF:

In [ ]:
peru_distritos=peru_distritos.assign(Educ_sec_comp2019_pct_lagged=ylag)

Plot both to compare:

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 8))

commonParams=dict(scheme="quantiles",cmap="Reds_r",legend=True)
# --- MAP 1
peru_distritos.plot("Educ_sec_comp2019_pct",ax=axes[0],**commonParams)
axes[0].set_title('Share Population HS Completed (original)', fontsize=14)
axes[0].set_axis_off()

# --- MAP 2
peru_distritos.plot("Educ_sec_comp2019_pct_lagged",ax=axes[1],**commonParams)
axes[1].set_title('Share Population HS Completed (lagged)', fontsize=14)
axes[1].set_axis_off()

plt.tight_layout()
plt.show()


In [ ]:
peru_distritos.plot.scatter("Educ_sec_comp2019_pct","Educ_sec_comp2019_pct_lagged")

In [ ]:
import esda

mi = esda.Moran(peru_distritos['Educ_sec_comp2019_pct'], peru_distritos_queen)

In [ ]:
mi.I,mi.p_sim

In [ ]:
lisa = esda.Moran_Local(peru_distritos['Educ_sec_comp2019_pct'], peru_distritos_queen)

In [ ]:
peru_distritos['cluster'] = lisa.get_cluster_labels(crit_value=0.05)

In [ ]:
peru_distritos['cluster'].value_counts()

In [ ]:
lisa.explore(peru_distritos,crit_value=0.05,
  prefer_canvas=True,
  tiles="CartoDB Positron",
)

## Global spatial correlation

If a spatial unit (a row) value in a variable is correlated with values of the neighbors, you know that proximity is interfering with the interpretation.

We need the neighboorhood matrix (the weight matrix) to compute spatial correlation.

If we standardize by row, the neighboors in a row add to 1:

In [ ]:
# needed for spatial correlation
w_knn8.transform = 'R'

Spatial correlation is measured by the Moran's I statistic:

In [ ]:
from esda.moran import Moran

moranHS = Moran(peru_distritos['Educ_sec_comp2019_pct'], w_knn8)
moranHS.I,moranHS.p_sim

A significant Moran's I suggest spatial correlation. Let's see the spatial scatter plot

In [ ]:
from splot.esda import moran_scatterplot

fig, ax = moran_scatterplot(moranHS, aspect_equal=True)
ax.set_xlabel('moranHS_std')
ax.set_ylabel('moranHS_lagged_std');

## Local Spatial Correlation

We can compute a Local Index of Spatial Association (LISA -local Moran) for each map object. That will help us find spatial clusters (spots) and spatial outliers:

* A **hotSpot** is a polygon whose value in the variable is high AND is surrounded with polygons with also high values.

* A **coldSpot** is a polygon whose value in the variable is low AND is surrounded with polygons with also low values.

* A **coldOutlier** is a polygon whose value in the variable is low BUT is surrounded with polygons with  high values.

* A **hotOutlier** is a polygon whose value in the variable is high BUT is surrounded with polygons with  low values.


High-High (HH): values above average surrounded by values above average.
Low-Low (LL): values below average surrounded by values below average.
High-Low (HL): values above average surrounded by values below average.
Low-High (LH): values below average surrounded by values above average.

It is also possible that no significant correlation is detected. Let's see those values:

In [ ]:
peru_distritos['cluster'].value_counts(sort=False)

In [ ]:

labels = [ '0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier']

lisaResults['HS_lisa_quadrant']=[labels[i] for i in lisaResults['HS_lisa']]

lisaResults['HS_lisa_quadrant'].value_counts()

In [ ]:
peru_distritos['HS_lisa_quadrant']=lisaResults['HS_lisa_quadrant']

In [ ]:
peru_distritos.crs

In [ ]:
import matplotlib.pyplot as plt
# custom colors
from matplotlib import colors
myColMap = colors.ListedColormap([ 'red', 'pink', 'snow', 'lightblue','blue'])

peru_distritos.plot(column='cluster',
                categorical=True,
                cmap=myColMap,
                linewidth=0.1,
                edgecolor='k',
                legend=True,
                legend_kwds={'bbox_to_anchor': (0.3, 0.3)},
                figsize=(12,12))


You find that a district is in a **quadrant**. If the district is NOT grey, then the LISA is significant. Let's represent that information in a map, using the lisaIDH object:

The info in **lisaIDH.q** can not be used right away, we need to add if the local spatial correlation is significant:

Now, we recode: